In [13]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
def bonbubun_allocator(x):
    # x is a string
    try:
        if pd.isna(x) == True:
            return x
        else:
            return str(int(x))
    except:
        return np.nan

In [3]:
def buildinggroup_modifier(x):
    # x is a string
    
    try:
        return int(x)
    except:
        return x

In [17]:
def read_coor_data1():
    basedir = './좌표데이터/'
    filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.txt'):
            try:
                df = pd.read_csv(basedir + filename, sep='|', header=None)#, usecols=[3,4,7,9,10,16,17])
            except:
                try:
                    df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='euc-kr')#, usecols=[3,4,7,9,10,16,17])
                except:
                    try:
                        df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='ansi')
                    except:
                        raise ValueError('myerror')
            
            return df
            
            df['도로명주소'] = df[3] + ' ' + df[4] + ' ' + df[7] + ' ' + df[9].apply(bonbubun_allocator) + '-' + df[10].apply(bonbubun_allocator)
            
            df['x좌표'] = df[16]
            df['y좌표'] = df[17]
            
            #df['건물군여부'] = df[14].apply(buildinggroup_modifier)
            
            df = df[['도로명주소', 'x좌표', 'y좌표']]
            
            #df['dtype'] = df['건물군여부'].apply(type)
            
            #df = df[df['dtype'] != str]
            
            #df.drop(columns=['dtype'], inplace=True)
                        
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).dropna().drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
        
    return concat_df

In [5]:
def read_coor_data2():
    basedir = './좌표데이터2/'
    filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.txt'):
            try:
                df = pd.read_csv(basedir + filename, sep='|', header=None, usecols=[1,2,5,7,8,23,24])
            except:
                df = pd.read_csv(basedir + filename, sep='|', header=None, encoding='ansi', usecols=[1,2,5,7,8,23,24])
                
            df['도로명주소'] = df[1] + ' ' + df[2] + ' ' + df[5] + ' ' + df[7].apply(bonbubun_allocator) + '-'\
            + df[8].apply(bonbubun_allocator)
            
            df['x좌표'] = df[23]
            df['y좌표'] = df[24]
            
            df = df[['도로명주소', 'x좌표', 'y좌표']]
            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).dropna().drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
        
    return concat_df

In [6]:
def read_coor_data3():
    basedir = './좌표데이터3/'
    filenames = [f for f in os.listdir(basedir) if f.endswith('.csv')]
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.csv'):
            df = pd.read_csv(basedir + filename, index_col=0)
            
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).dropna().drop_duplicates(subset=['전체주소'], keep='last').reset_index(drop=True)
    concat_df['지번주소'] = concat_df['전체주소'].copy()
    concat_df['도로명주소'] = concat_df['전체주소'].copy()
    concat_df = concat_df[['지번주소', '도로명주소', 'x좌표', 'y좌표']]
    
    return concat_df

In [7]:
def add_0(x):
    if pd.isnull(x) == False:
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x
    else:
        return x

In [8]:
def read_coor_data4():
    basedir = './좌표데이터4/'
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.txt'))&(f.startswith('match_jibun'))]
        
    colnames = ['법정동코드', '시도명', '시군구명', '읍면동명', '리명', '산여부', '지번본번(번지)', '지번부번(호)', '도로명코드',
                '지하여부', '건물본번', '건물부번', '지번일련번호', '시도명(영문)', '시군구명(영문)', '읍면동명(영문)', '리명(영문)',
                '이동사유코드', '건물관리번호', '주소관할읍명동코드'
               ]
    
    dfs_list = []
    for file in tqdm(filenames):
        df = pd.read_csv(basedir + file, sep='|', header=None, names=colnames, encoding='euc-kr')
        dfs_list.append(df)
        
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [9]:
def read_coor_data5():
    basedir = './좌표데이터4/'
    filenames = [f for f in os.listdir(basedir) if (f.startswith('match_build')&(f.endswith('.txt')))]
    
    colnames = ['주소관할읍면동코드', '시도명', '시군구명', '읍면동명', '도로명코드', '도로명', '지하여부', '건물본번', '건물부번',
                '우편번호', '건물관리번호', '시군구용건물명', '건축물용도분류', '행정동코드', '행정동명', '지상층수', '지하층수',
                '공동주택구분', '건물수', '상세건물명', '건물명변경이력', '상세건물명변경이력', '거주여부', '건물중심점_x좌표',
                '건물중심점_y좌표', '출입구_x좌표', '출입구_y좌표', '시도명(영문)', '시군구명(영문)', '읍면동명(영문)',
                '도로명(영문)', '읍면동구분', '이동사유코드']
    
    dfs_list = []
    for file in filenames:
        df = pd.read_csv(basedir + file, sep='|', header=None, encoding='ansi', names=colnames)
        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [10]:
prac_coor_df = read_coor_data4()
print(prac_coor_df.shape)
prac_coor_df.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.56s/it]


(641851, 20)


,법정동코드,시도명,시군구명,읍면동명,리명,산여부,지번본번(번지),지번부번(호),도로명코드,지하여부,건물본번,건물부번,지번일련번호,시도명(영문),시군구명(영문),읍면동명(영문),리명(영문),이동사유코드,건물관리번호,주소관할읍명동코드
0,1111011900,서울특별시,종로구,세종로,NaN,0,215,0,111102005001,0,145,2,0,Seoul,Jongno-gu,Sejongno,NaN,NaN,1111011900102150000000001,1111011900
1,1111011900,서울특별시,종로구,세종로,NaN,0,1,80,111102005001,0,147,2,0,Seoul,Jongno-gu,Sejongno,NaN,NaN,1111011900100010080000001,1111011900
2,1111011900,서울특별시,종로구,세종로,NaN,0,211,0,111102005001,0,149,0,0,Seoul,Jongno-gu,Sejongno,NaN,NaN,1111012000101500000000347,1111011900
3,1111012000,서울특별시,종로구,신문로1가,NaN,0,150,0,111102005001,0,149,0,1114,Seoul,Jongno-gu,Sinmunno 1(il)-ga,NaN,NaN,1111012000101500000000347,1111011900
4,1114010300,서울특별시,중구,태평로1가,NaN,0,68,0,111102005001,0,149,0,10238,Seoul,Jung-gu,Taepyeongno 1(il)-ga,NaN,NaN,1111012000101500000000347,1111011900


In [11]:
prac_coor_df['읍면동명'].value_counts()

신림동       20642
화곡동       14792
봉천동       14684
미아동       13300
면목동       12756
수유동       11512
신길동        9408
구로동        9371
상계동        9250
중곡동        8153
상도동        7261
방배동        7237
독산동        7225
신월동        7196
사당동        6987
장위동        6856
시흥동        6768
자양동        6737
신당동        6605
정릉동        6566
천호동        6435
신사동        5942
이문동        5431
역삼동        5390
불광동        5344
신정동        5298
대림동        5233
목동         5107
응암동        5066
논현동        4925
구의동        4837
개봉동        4786
갈현동        4766
망우동        4705
쌍문동        4591
서초동        4478
창동         4461
제기동        4370
홍은동        4339
연희동        4305
중화동        4296
성내동        4282
홍제동        4210
방학동        3877
전농동        3847
방화동        3723
석관동        3662
역촌동        3661
장안동        3630
이태원동       3582
노량진동       3556
번동         3495
창신동        3441
한남동        3395
용두동        3390
성수동2가      3320
도봉동        3262
마천동        3171
묵동         3138
공릉동        3128
상봉동        3102
망원동        3099
북가좌동    

In [18]:
prac_coor_df2 = read_coor_data1()

  0%|                                                                                           | 0/26 [00:01<?, ?it/s]


In [19]:
prac_coor_df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,11110,760,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,94,0,NaN,3047,근린생활시설,0,청운효자동,953241.683263,1.954023e+06
1,11110,4933,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,96,0,평안빌,3047,주택,0,청운효자동,953243.258151,1.954034e+06
2,11110,287,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,98,0,청운빌라,3047,주택,0,청운효자동,953240.877646,1.954055e+06
3,11110,1336,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,99,3,풍림팍사이드빌라,3032,주택,1,청운효자동,953195.459364,1.954079e+06
4,11110,6021,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,99,4,경복빌라,3032,주택,1,청운효자동,953193.346779,1.954097e+06


In [ ]:
prac_coor_df = prac_coor_df[['시도명', '시군구명', '읍면동명', '산여부', '지번본번(번지)', '지번부번(호)']]

In [17]:
prac_coor_df2 = read_coor_data5()
print(prac_coor_df2.shape)
prac_coor_df2.head()

(626925, 33)


,주소관할읍면동코드,시도명,시군구명,읍면동명,도로명코드,도로명,지하여부,건물본번,건물부번,우편번호,건물관리번호,시군구용건물명,건축물용도분류,행정동코드,행정동명,지상층수,지하층수,공동주택구분,건물수,상세건물명,건물명변경이력,상세건물명변경이력,거주여부,건물중심점_x좌표,건물중심점_y좌표,출입구_x좌표,출입구_y좌표,시도명(영문),시군구명(영문),읍면동명(영문),도로명(영문),읍면동구분,이동사유코드
0,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,94,0,3047,1111010100101440003031291,NaN,근린생활시설,1.111052e+09,청운효자동,1,0,0,1,NaN,NaN,NaN,0,953243.013280,1.954026e+06,953241.683263,1.954023e+06,Seoul,Jongno-gu,Cheongun-dong,Jahamun-ro,1,NaN
1,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,96,0,3047,1111010100101080014031432,평안빌,주택,1.111052e+09,청운효자동,4,0,2,1,NaN,NaN,NaN,1,953247.695696,1.954041e+06,953243.258151,1.954034e+06,Seoul,Jongno-gu,Cheongun-dong,Jahamun-ro,1,NaN
2,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,98,0,3047,1111010100101080013031431,청운빌라,주택,1.111052e+09,청운효자동,4,1,2,1,에이동,NaN,NaN,1,953248.552199,1.954060e+06,953240.877646,1.954055e+06,Seoul,Jongno-gu,Cheongun-dong,Jahamun-ro,1,NaN
3,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,99,3,3032,1111010100101340002031304,풍림팍사이드빌라,주택,1.111052e+09,청운효자동,4,1,2,2,NaN,NaN,NaN,1,953194.413607,1.954068e+06,953195.459364,1.954079e+06,Seoul,Jongno-gu,Cheongun-dong,Jahamun-ro,1,NaN
4,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,99,3,3032,1111010100101340002031311,풍림팍사이드빌라,주택,1.111052e+09,청운효자동,0,0,2,2,NaN,NaN,NaN,1,953183.480942,1.954081e+06,953195.459364,1.954079e+06,Seoul,Jongno-gu,Cheongun-dong,Jahamun-ro,1,NaN


In [8]:
def read_coor_data():
    coor_df1 = read_coor_data1()
    coor_df2 = read_coor_data2()
    coor_df3 = read_coor_data3()    
    
    coor_df = pd.concat([coor_df1, coor_df2, coor_df3]).drop_duplicates(subset=['도로명주소'], keep='last').reset_index(drop=True)
    coor_df = coor_df[['지번주소', '도로명주소', 'x좌표', 'y좌표']]    
            
    return coor_df

In [9]:
coor_df = read_coor_data()
print(coor_df.shape)
coor_df.head()

100%|█████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 111.49it/s]
c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


(657309, 4)


,지번주소,도로명주소,x좌표,y좌표
0,NaN,서울특별시 관악구 난곡로24가길 18-0,948851.369529,1.941318e+06
1,NaN,서울특별시 종로구 성균관로15길 33-0,955591.635372,1.954533e+06
2,NaN,서울특별시 성북구 인촌로7길 70-0,957563.072605,1.954475e+06
3,NaN,서울특별시 서초구 강남대로91길 5-0,957617.535913,1.945960e+06
4,NaN,서울특별시 서초구 바우뫼로11길 54-0,958121.727726,1.941641e+06


In [11]:
main_data_df = pd.read_csv('./prepped_data/main_data_ver_2.2.csv')
print(main_data_df.shape)
main_data_df.head()

(4559866, 27)


,시군구,번지,본번,부번,건물명,전월세매매구분,전용면적(㎡),계약년월,계약일,금액(만원),월세(만원),층,건축년도,도로명,가격/면적,year,month,시,구,target_log_transformed,target/area_log_transformed,동,건물종류,trade_type,지번주소,대장구분코드,대장구분명
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201309,8,57000,0,2,1988,언주로 103,733.118971,2013,9,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),매매,77.75,201312,16,57000,0,2,1988,언주로 103,733.118971,2013,12,서울특별시,강남구,10.950807,6.597308,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 655-2,1,일반
2,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,11,55000,0,5,1987,언주로 3,817.479191,2013,2,서울특별시,강남구,10.915088,6.706225,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반
3,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201302,22,58250,0,4,1987,언주로 3,865.784780,2013,2,서울특별시,강남구,10.972499,6.763636,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반
4,서울특별시 강남구 개포동,658-1,658,1,개포6차우성아파트1동~8동,매매,67.28,201305,10,60000,0,5,1987,언주로 3,891.795482,2013,5,서울특별시,강남구,11.002100,6.793237,서울특별시 강남구 개포동,아파트,아파트_매매,서울특별시 강남구 개포동 658-1,1,일반


In [12]:
def roadname_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        splitted = x.split('-')
        
        if len(splitted) == 1:
            return x+'-0'
        elif len(splitted) == 2:
            return x
        else:
            raise ValueError('myerror')
            

In [13]:
main_data_df['도로명주소'] = main_data_df['시']+' '+main_data_df['구']+' '+main_data_df['도로명'].apply(roadname_modifier)

In [14]:
#main_data_df.to_csv('main_data_ver_2.3.csv', index=False)

In [13]:
selected_df = main_data_df[(main_data_df['지번주소'].isin(coor_df['지번주소'].unique()))
                           |(main_data_df['지번주소'].isin(coor_df['도로명주소'].unique()))
                          ]

In [14]:
no_jibun_df = main_data_df.drop(selected_df.index)

In [15]:
reselected_df = no_jibun_df[(no_jibun_df['도로명주소'].isin(coor_df['지번주소'].unique()))
                       |(no_jibun_df['도로명주소'].isin(coor_df['도로명주소'].unique()))
                      ]

In [16]:
no_coor_df = no_jibun_df.drop(reselected_df.index)

In [18]:
no_coor_df2 = pd.DataFrame({'지번주소':no_coor_df['지번주소'].unique().tolist()})

In [19]:
no_coor_df2.to_csv('no_coor_addrs.csv', index=False)

In [20]:
coor_df.head()

,지번주소,도로명주소,x좌표,y좌표
0,NaN,서울특별시 관악구 난곡로24가길 18-0,948851.369529,1.941318e+06
1,NaN,서울특별시 종로구 성균관로15길 33-0,955591.635372,1.954533e+06
2,NaN,서울특별시 성북구 인촌로7길 70-0,957563.072605,1.954475e+06
3,NaN,서울특별시 서초구 강남대로91길 5-0,957617.535913,1.945960e+06
4,NaN,서울특별시 서초구 바우뫼로11길 54-0,958121.727726,1.941641e+06


In [22]:
selected_df = coor_df[(coor_df['지번주소'].isin(main_data_df['지번주소'].unique()))
                      |(coor_df['지번주소'].isin(main_data_df['도로명주소'].unique()))
                     ]

In [23]:
no_jibun_df = coor_df.drop(selected_df.index)

In [24]:
reselected_df = no_jibun_df[(no_jibun_df['도로명주소'].isin(main_data_df['지번주소'].unique()))
                       |(no_jibun_df['도로명주소'].isin(main_data_df['도로명주소'].unique()))
                      ]

In [26]:
no_addr_df = no_jibun_df.drop(reselected_df.index)
print(no_addr_df.shape)

(6274, 4)


In [27]:
no_addr_df.head()

,지번주소,도로명주소,x좌표,y좌표
566442,서울특별시 서초구 고무래로,서울특별시 서초구 고무래로,956676.746259,1.945089e+06
566448,서울특별시 강남구 260,서울특별시 강남구 260,960294.041765,1.942320e+06
566450,서울특별시 마포구 망원동 521,서울특별시 마포구 망원동 521,947232.206233,1.950512e+06
566451,서울특별시 강남구 개포로,서울특별시 강남구 개포로,962615.409421,1.943838e+06
566452,서울특별시 강남구 310,서울특별시 강남구 310,958582.891361,1.943629e+06


In [30]:
print(coor_df.shape)
coor_df.drop(no_addr_df.index, inplace=True)
coor_df.shape

(657309, 4)


(651035, 4)

In [31]:
coor_df.to_csv('coor_data_ver_1.csv', index=False)